# Excercise Detection

Este notebook recogerá el proceso que sigue todo el modelo desde que recibe un video hasta que detecta el tipo de ejercicio que se está haciendo en el video

**Importante:** En este notebook no se recoge la creación de la corrección de posturas. Este paso se integra en el archivo _excercise\_classification.py_ donde ya se integra el modelo LLM con Ollama, y se crean las funciones para detectar posturas en base a los datos de videos

## Imports 

In [1]:
import os
import cv2
import time
import urllib
import pickle
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from PIL import Image
from IPython.display import Image as IPyImage
#from ultralytics import YOLO
from matplotlib import pyplot as plt
from keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from collections import Counter
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
workpath = 'C:/Users/Legion/TFM/Tareas'
os.chdir(workpath)
import warnings
warnings.filterwarnings("ignore")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:/Users/Legion/TFM/Tareas'

## Function definition

In [4]:

def create_angle(a, b, c):
    A = np.array(a)
    B = np.array(b)
    C = np.array(c)
    # Vectors BA y BC
    BA = A - B
    BC = C - B
    # Dot product between vectors BA y BC
    dot_product = np.dot(BA, BC)
    # Vector norms BA y BC
    magnitude_BA = np.linalg.norm(BA)
    magnitude_BC = np.linalg.norm(BC)
    # Angle between vectors' cosine
    cos_theta = dot_product / (magnitude_BA * magnitude_BC)
    # Angle in radions
    theta_radians = np.arccos(cos_theta)
    # Angle in degrees
    theta_degrees = np.degrees(theta_radians)
    return theta_degrees




def retrieve_data_video(path):
    cap = cv2.VideoCapture(path)
    landmarks_raw = []
    angles = {'elbow_l':[], 'elbow_r':[],
              'shoulder_l': [], 'shoulder_r':[],
              'hip_l': [], 'hip_r': [],
              'knee_l': [], 'knee_r': []}
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.8 , min_tracking_confidence=0.8) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
        
            # Recolor image to RGB
            if ret == True:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
          
                # Make detection
                results = pose.process(image)
                try:
                    landmarks_frame = results.pose_landmarks.landmark
                    landmarks_raw.append(landmarks_frame)
                    left_wrist = [landmarks_frame[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks_frame[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                                  landmarks_frame[mp_pose.PoseLandmark.LEFT_WRIST.value].z]
                    left_elbow = [landmarks_frame[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks_frame[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                                  landmarks_frame[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
                    left_shoulder = [landmarks_frame[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks_frame[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                     landmarks_frame[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                    left_hip = [landmarks_frame[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks_frame[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                                landmarks_frame[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                    left_knee = [landmarks_frame[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks_frame[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                                 landmarks_frame[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                    left_ankle = [landmarks_frame[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks_frame[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
                                  landmarks_frame[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                    right_wrist = [landmarks_frame[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks_frame[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                                  landmarks_frame[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]
                    right_elbow = [landmarks_frame[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks_frame[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                                  landmarks_frame[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]
                    right_shoulder = [landmarks_frame[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks_frame[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                     landmarks_frame[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                    right_hip = [landmarks_frame[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks_frame[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                 landmarks_frame[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                    right_knee = [landmarks_frame[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks_frame[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                  landmarks_frame[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                    right_ankle = [landmarks_frame[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks_frame[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                   landmarks_frame[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
                    
                    angles['elbow_l'].append(create_angle(left_wrist, left_elbow, left_shoulder))
                    angles['elbow_r'].append(create_angle(right_wrist, right_elbow, right_shoulder))
                    angles['shoulder_l'].append(create_angle(left_elbow, left_shoulder, left_hip))
                    angles['shoulder_r'].append(create_angle(right_elbow, right_shoulder, right_hip))
                    angles['hip_l'].append(create_angle(left_shoulder, left_hip, left_knee))
                    angles['hip_r'].append(create_angle(right_shoulder, right_hip, right_knee))
                    angles['knee_l'].append(create_angle(left_hip, left_knee, left_ankle))
                    angles['knee_r'].append(create_angle(right_hip, right_knee, right_ankle))
                except Exception as e:
                    print(e)
                    pass
                # Recolor back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                # Render detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(148,7,240), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(149,158,0), thickness=2, circle_radius=2) 
                                         )               
        
                cv2.imshow('Mediapipe Feed', image)

                if cv2.waitKey(25) == ord('q'):
                    break
            else:
                break
        cap.release()
        cv2.destroyAllWindows()
    return landmarks_raw, angles

def landmarks_to_dataframe(rawdata, angles):
    landmarks_dict = {
    'NOSE_x' : [],
    'NOSE_y' : [],
    'NOSE_z' : [],
    'LEFT_SHOULDER_x' : [],
    'LEFT_SHOULDER_y' : [],
    'LEFT_SHOULDER_z' : [],
    'RIGHT_SHOULDER_x' : [],
    'RIGHT_SHOULDER_y' : [],
    'RIGHT_SHOULDER_z' : [],
    'LEFT_ELBOW_x' : [],
    'LEFT_ELBOW_y' : [],
    'LEFT_ELBOW_z' : [],
    'RIGHT_ELBOW_x' : [],
    'RIGHT_ELBOW_y' : [],
    'RIGHT_ELBOW_z' : [],
    'LEFT_WRIST_x' : [],
    'LEFT_WRIST_y' : [],
    'LEFT_WRIST_z' : [],
    'RIGHT_WRIST_x' : [],
    'RIGHT_WRIST_y' : [],
    'RIGHT_WRIST_z' : [],
    'LEFT_PINKY_x' : [],
    'LEFT_PINKY_y' : [],
    'LEFT_PINKY_z' : [],
    'RIGHT_PINKY_x' : [],
    'RIGHT_PINKY_y' : [],
    'RIGHT_PINKY_z' : [],
    'LEFT_INDEX_x' : [],
    'LEFT_INDEX_y' : [],
    'LEFT_INDEX_z' : [],
    'RIGHT_INDEX_x' : [],
    'RIGHT_INDEX_y' : [],
    'RIGHT_INDEX_z' : [],
    'LEFT_THUMB_x' : [],
    'LEFT_THUMB_y' : [],
    'LEFT_THUMB_z' : [],
    'RIGHT_THUMB_x' : [],
    'RIGHT_THUMB_y' : [],
    'RIGHT_THUMB_z' : [],
    'LEFT_HIP_x' : [],
    'LEFT_HIP_y' : [],
    'LEFT_HIP_z' : [],
    'RIGHT_HIP_x' : [],
    'RIGHT_HIP_y' : [],
    'RIGHT_HIP_z' : [],
    'LEFT_KNEE_x' : [],
    'LEFT_KNEE_y' : [],
    'LEFT_KNEE_z' : [],
    'RIGHT_KNEE_x' : [],
    'RIGHT_KNEE_y' : [],
    'RIGHT_KNEE_z' : [],
    'LEFT_ANKLE_x' : [],
    'LEFT_ANKLE_y' : [],
    'LEFT_ANKLE_z' : [],
    'RIGHT_ANKLE_x' : [],
    'RIGHT_ANKLE_y' : [],
    'RIGHT_ANKLE_z' : [],
    'LEFT_ELBOW_ANGLE': [],
    'RIGHT_ELBOW_ANGLE': [],
    'LEFT_SHOULDER_ANGLE': [],
    'RIGHT_SHOULDER_ANGLE': [],
    'LEFT_HIP_ANGLE': [],
    'RIGHT_HIP_ANGLE': [],
    'LEFT_KNEE_ANGLE': [],
    'RIGHT_KNEE_ANGLE': []
    }
    for index in range(len(rawdata)):
        landmarks_dict['NOSE_x'].append(rawdata[index][mp_pose.PoseLandmark.NOSE.value].x)
        landmarks_dict['NOSE_y'].append(1-rawdata[index][mp_pose.PoseLandmark.NOSE.value].y)
        landmarks_dict['NOSE_z'].append(rawdata[index][mp_pose.PoseLandmark.NOSE.value].z)
        landmarks_dict['LEFT_SHOULDER_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_SHOULDER.value].x)
        landmarks_dict['LEFT_SHOULDER_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
        landmarks_dict['LEFT_SHOULDER_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_SHOULDER.value].z)
        landmarks_dict['RIGHT_SHOULDER_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x)
        landmarks_dict['RIGHT_SHOULDER_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
        landmarks_dict['RIGHT_SHOULDER_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z)
        landmarks_dict['LEFT_ELBOW_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_ELBOW.value].x)
        landmarks_dict['LEFT_ELBOW_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_ELBOW.value].y)
        landmarks_dict['LEFT_ELBOW_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_ELBOW.value].z)
        landmarks_dict['RIGHT_ELBOW_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_ELBOW.value].x)
        landmarks_dict['RIGHT_ELBOW_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_ELBOW.value].y)
        landmarks_dict['RIGHT_ELBOW_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_ELBOW.value].z)
        landmarks_dict['LEFT_WRIST_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_WRIST.value].x)
        landmarks_dict['LEFT_WRIST_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_WRIST.value].y)
        landmarks_dict['LEFT_WRIST_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_WRIST.value].z)
        landmarks_dict['RIGHT_WRIST_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_WRIST.value].x)
        landmarks_dict['RIGHT_WRIST_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
        landmarks_dict['RIGHT_WRIST_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_WRIST.value].z)
        landmarks_dict['LEFT_PINKY_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_PINKY.value].x)
        landmarks_dict['LEFT_PINKY_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_PINKY.value].y)
        landmarks_dict['LEFT_PINKY_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_PINKY.value].z)
        landmarks_dict['RIGHT_PINKY_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_PINKY.value].x)
        landmarks_dict['RIGHT_PINKY_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_PINKY.value].y)
        landmarks_dict['RIGHT_PINKY_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_PINKY.value].z)
        landmarks_dict['LEFT_INDEX_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_INDEX.value].x)
        landmarks_dict['LEFT_INDEX_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_INDEX.value].y)
        landmarks_dict['LEFT_INDEX_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_INDEX.value].z)
        landmarks_dict['RIGHT_INDEX_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_INDEX.value].x)
        landmarks_dict['RIGHT_INDEX_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_INDEX.value].y)
        landmarks_dict['RIGHT_INDEX_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_INDEX.value].z)
        landmarks_dict['LEFT_THUMB_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_THUMB.value].x)
        landmarks_dict['LEFT_THUMB_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_THUMB.value].y)
        landmarks_dict['LEFT_THUMB_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_THUMB.value].z)
        landmarks_dict['RIGHT_THUMB_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_THUMB.value].x)
        landmarks_dict['RIGHT_THUMB_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_THUMB.value].y)
        landmarks_dict['RIGHT_THUMB_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_THUMB.value].z)
        landmarks_dict['LEFT_HIP_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_HIP.value].x)
        landmarks_dict['LEFT_HIP_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_HIP.value].y)
        landmarks_dict['LEFT_HIP_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_HIP.value].z)
        landmarks_dict['RIGHT_HIP_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_HIP.value].x)
        landmarks_dict['RIGHT_HIP_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_HIP.value].y)
        landmarks_dict['RIGHT_HIP_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_HIP.value].z)
        landmarks_dict['LEFT_KNEE_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_KNEE.value].x)
        landmarks_dict['LEFT_KNEE_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_KNEE.value].y)
        landmarks_dict['LEFT_KNEE_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_KNEE.value].z)
        landmarks_dict['RIGHT_KNEE_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_KNEE.value].x)
        landmarks_dict['RIGHT_KNEE_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_KNEE.value].y)
        landmarks_dict['RIGHT_KNEE_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_KNEE.value].z)
        landmarks_dict['LEFT_ANKLE_x'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_ANKLE.value].x)
        landmarks_dict['LEFT_ANKLE_y'].append(1-rawdata[index][mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
        landmarks_dict['LEFT_ANKLE_z'].append(rawdata[index][mp_pose.PoseLandmark.LEFT_ANKLE.value].z)
        landmarks_dict['RIGHT_ANKLE_x'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_ANKLE.value].x)
        landmarks_dict['RIGHT_ANKLE_y'].append(1-rawdata[index][mp_pose.PoseLandmark.RIGHT_ANKLE.value].y)
        landmarks_dict['RIGHT_ANKLE_z'].append(rawdata[index][mp_pose.PoseLandmark.RIGHT_ANKLE.value].z)
        landmarks_dict['LEFT_ELBOW_ANGLE'].append(angles['elbow_l'][index])
        landmarks_dict['RIGHT_ELBOW_ANGLE'].append(angles['elbow_r'][index])
        landmarks_dict['LEFT_SHOULDER_ANGLE'].append(angles['shoulder_l'][index])
        landmarks_dict['RIGHT_SHOULDER_ANGLE'].append(angles['shoulder_r'][index])
        landmarks_dict['LEFT_HIP_ANGLE'].append(angles['hip_l'][index])
        landmarks_dict['RIGHT_HIP_ANGLE'].append(angles['hip_r'][index])
        landmarks_dict['LEFT_KNEE_ANGLE'].append(angles['knee_l'][index])
        landmarks_dict['RIGHT_KNEE_ANGLE'].append(angles['knee_r'][index])
    return pd.DataFrame(landmarks_dict)
    
def minmaxing_rows(df):
    standarized_df = df.subtract(
        df.min(axis=1), axis=0).divide(
        df.max(axis=1) - df.min(axis=1), axis=0).combine_first(df)
    return standarized_df


def standarize_coordinates(dataframe):
    sorted_cols = list(dataframe.columns)
    x_coord = dataframe[[xcol for xcol in dataframe.columns if '_x' in xcol]]
    y_coord = dataframe[[ycol for ycol in dataframe.columns if '_y' in ycol]]
    z_coord = dataframe[[zcol for zcol in dataframe.columns if '_z' in zcol]]
    angles = dataframe[[anglecol for anglecol in dataframe.columns if '_ANGLE' in anglecol]]
    
    x_coord_ref = x_coord.sub(x_coord['NOSE_x'], axis=0)
    y_coord_ref = y_coord.sub(y_coord['NOSE_y'], axis=0)
    z_coord_ref = z_coord.sub(z_coord['NOSE_z'], axis=0)

    minmaxed_x_coord = minmaxing_rows(x_coord_ref)
    minmaxed_y_coord = minmaxing_rows(y_coord_ref)
    minmaxed_z_coord = minmaxing_rows(z_coord_ref)

    df_scaled_xy = minmaxed_x_coord.join(minmaxed_y_coord, how='inner')
    df_scaled_xyz = df_scaled_xy.join(minmaxed_z_coord, how='inner')
    df_scaled = df_scaled_xyz.join(angles, how='inner')
    
    return df_scaled[sorted_cols]
#.apply(lambda x : round(x, 2)).drop_duplicates(subset=['NOSE_x', 'NOSE_y'])

def create_sequences(df):
    sequences = []
    sequence_length = 100
    variables = list(df.columns)
    for i in range(len(df) - sequence_length + 1):
        seq = df.iloc[i:i + sequence_length][variables].values
        sequences.append(seq)
    return np.array(sequences)

def run_model(sequences):
    model = load_model('C:/Users/Legion/TFM/Tareas/Models/classification_model_seq100.keras')
    y_pred = model.predict(sequences)
    return y_pred


def excercise_from_prediction(y_pred):
    preds = [np.argmax(x) for x in y_pred]
    preds_no_rest = [x for x in preds if x!=4]
    count_preds = Counter(preds)
    final_pred_num = count_preds.most_common(1)[0]
    num_to_excercise = {0: 'Dominada', 1: 'Fondo',
                        2: 'Flexión', 3: 'Sentadilla', 
                        4: 'Descanso'}
    predicted_excercise = num_to_excercise[final_pred_num[0]]
    return predicted_excercise


def pipeline_video_to_excercise(video_path: str):
    landmarks_video, angles = retrieve_data_video(video_path)
    df_landmarks = landmarks_to_dataframe(landmarks_video, angles)
    df_standarized_landmarks = standarize_coordinates(df_landmarks)
    sequences_video = create_sequences(df_standarized_landmarks)
    prediction_from_model = run_model(sequences_video)
    excercise = excercise_from_prediction(prediction_from_model)
    print(f'El ejercicio que estás haciendo es: {excercise}')
    return excercise

def knn_model(data_model):
    with open('C:/Users/Legion/TFM/Tareas/Models/classification_model_knn3.pickle', 'rb') as file:
        knn_model = pickle.load(file)
    y_pred = knn_model.predict(data_model)
    return y_pred

def excercise_from_prediction_knn(y_pred):
    count_preds = Counter(y_pred)
    print(y_pred)
    print(count_preds)
    final_pred_num = count_preds.most_common(1)[0]
    num_to_excercise = {0: 'Dominada', 1: 'Fondo',
                        2: 'Flexión', 3: 'Sentadilla', 
                        4: 'Stand Up'}
    predicted_excercise = num_to_excercise[final_pred_num[0]]

    return predicted_excercise
    
def pipeline_classification_knn(video_path: str):
    landmarks_video, angles = retrieve_data_video(video_path)
    df_landmarks = landmarks_to_dataframe(landmarks_video, angles)
    df_standarized_landmarks = standarize_coordinates(df_landmarks)
    prediction_from_model = knn_model(df_standarized_landmarks)
    excercise = excercise_from_prediction_knn(prediction_from_model)
    print(f'El ejercicio que estás haciendo es: {excercise}')
    return prediction_from_model, df_standarized_landmarks

In [5]:
import time

# Comienza a medir el tiempo
start_time = time.time()

# El comando que deseas medir
result = pipeline_video_to_excercise('C:/Users/Legion/TFM/Tareas/VideosTR/PruebasDeModelo/SentadillaPr.mp4')

# Termina de medir el tiempo
end_time = time.time()

# Calcula el tiempo transcurrido
elapsed_time = end_time - start_time

print(f"El comando tardó {elapsed_time} segundos en ejecutarse.")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 294ms/step
[[9.99986410e-01 1.65989672e-06 1.63208472e-06 1.01249734e-05
  1.63761428e-07]
 [9.99986768e-01 1.75416608e-06 1.61619062e-06 9.65438539e-06
  1.52436783e-07]
 [9.99987602e-01 1.85965632e-06 1.57828470e-06 8.87406441e-06
  1.41714381e-07]
 [9.99987602e-01 1.94020004e-06 1.54370707e-06 8.76971535e-06
  1.36540379e-07]
 [9.99988079e-01 2.03206650e-06 1.54962117e-06 8.20083460e-06
  1.27798046e-07]
 [9.99987125e-01 2.05026936e-06 1.48309482e-06 9.24709275e-06
  1.29805727e-07]
 [9.99985933e-01 2.13230601e-06 1.43345767e-06 1.03876173e-05
  1.30208278e-07]
 [9.99984503e-01 2.20669699e-06 1.44860132e-06 1.17886584e-05
  1.45306956e-07]
 [9.99983430e-01 2.26578027e-06 1.44738453e-06 1.28014935e-05
  1.61891663e-07]
 [9.99982476e-01 2.33204355e-06 1.39349675e-06 1.37009793e-05
  1.66432784e-07]
 [9.99979973e-01 2.39531323e-06 1.38193320e-06 1.59641331e-05
  1.84291650e-07]
 [9.99978900e-01 2.50241965e-06 1.34574157e-06 1.70829680e-05
  1.86728627e-07]
 

In [39]:
import time

# Comienza a medir el tiempo
start_time = time.time()

# Ejecutar pipeline
result, df_estandar = pipeline_classification_knn('C:/Users/Legion/TFM/Tareas/VideosTR/PruebasDeModelo/SentadillaPr.mp4')

# Termina de medir el tiempo
end_time = time.time()

# Calcula el tiempo transcurrido
elapsed_time = end_time - start_time

print(f"El comando tardó {elapsed_time} segundos en ejecutarse.")

[0 0 0 0 0 0 0 0 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 4 0 4 1 2 0 4 1 1 4 4 4
 3 3 1 3 4]
Counter({0: 56, 3: 11, 4: 7, 1: 4, 2: 1})
El ejercicio que estás haciendo es: Dominada
El comando tardó 12.538768529891968 segundos en ejecutarse.


In [5]:
result

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 1, 2, 0, 4, 1, 1, 4, 0, 1, 4, 4, 4, 4, 3, 3, 3, 1, 1, 3, 3, 4, 4, 4], dtype=int64)

In [12]:
retrieve_data_video('C:/Users/Legion/TFM/Tareas/VideosTR/PruebasDeModelo/Sentadillas2.mp4')

([],
 {'elbow_l': [],
  'elbow_r': [],
  'shoulder_l': [],
  'shoulder_r': [],
  'hip_l': [],
  'hip_r': [],
  'knee_l': [],
  'knee_r': []})

# Working with data for pose correction

### Main problem

Una vez tenemos un modelo que clasifica correctamente los distintos tipos de ejercicios, asumiremos que los videos se procesan correctamente (bastantes fallos de pose tracking fuera de nuestro alcance para resolver) y en base a los puntos clave recogidos por el modelo y los ángulos de las conexiones más importantes habrá que estudiar cómo verificar si el ejercicio se está haciendo correctamente o no.

### Estructura de datos

In [23]:
df_estandar

,NOSE_x,NOSE_y,NOSE_z,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,RIGHT_SHOULDER_z,LEFT_ELBOW_x,...,RIGHT_ANKLE_y,RIGHT_ANKLE_z,LEFT_ELBOW_ANGLE,RIGHT_ELBOW_ANGLE,LEFT_SHOULDER_ANGLE,RIGHT_SHOULDER_ANGLE,LEFT_HIP_ANGLE,RIGHT_HIP_ANGLE,LEFT_KNEE_ANGLE,RIGHT_KNEE_ANGLE
0,0.548453,1.000000,0.733806,0.909367,0.948681,0.545027,0.778652,0.861502,0.926796,0.691304,...,0.027365,0.241823,132.026961,80.591739,27.867744,121.683314,167.602795,171.902782,169.780329,174.431579
1,0.375722,0.989541,0.680184,0.854227,1.000000,0.465414,0.681092,0.901958,0.885117,0.661388,...,0.028111,0.317934,137.064235,65.356284,44.527388,125.765890,146.058917,170.088839,164.967431,174.107698
2,0.358369,0.987619,0.693668,0.839920,1.000000,0.490861,0.646468,0.897857,0.884002,0.665834,...,0.027351,0.285854,137.215561,60.991021,39.765996,127.932746,149.692358,169.614377,164.062154,174.571791
3,0.349189,0.990606,0.672986,0.819051,1.000000,0.452218,0.658313,0.904318,0.885308,0.657978,...,0.034068,0.341383,141.004281,59.789135,49.270789,124.412488,140.954901,167.931648,162.315923,172.425646
4,0.343173,0.990643,0.688125,0.817446,1.000000,0.493202,0.653829,0.902292,0.879020,0.656315,...,0.037293,0.277140,138.650176,62.133701,41.582489,130.693700,146.654252,169.953070,165.441873,173.668652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.642401,1.000000,0.622060,0.799856,0.933690,0.365944,0.805046,0.867816,0.874688,0.418716,...,0.020487,0.373867,114.853987,107.596553,86.561001,135.391474,138.771219,160.465377,160.580879,171.515148
502,0.643541,1.000000,0.588152,0.812467,0.938038,0.321686,0.809195,0.871052,0.859379,0.421842,...,0.026942,0.376507,112.267847,111.150830,86.743401,137.095200,142.410854,159.902013,163.638445,167.232719
503,0.647605,1.000000,0.627487,0.830095,0.938307,0.382493,0.791101,0.858339,0.889559,0.452206,...,0.031596,0.357077,127.615429,109.059801,76.973778,134.362742,146.718228,164.446560,163.652520,168.275623
504,0.648450,1.000000,0.621049,0.832269,0.934036,0.378477,0.803137,0.861849,0.880209,0.466918,...,0.034429,0.355969,124.301118,108.103146,76.472687,134.713300,146.230045,164.430482,162.672965,167.937284


In [25]:
df_estandar.describe()

,NOSE_x,NOSE_y,NOSE_z,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,RIGHT_SHOULDER_z,LEFT_ELBOW_x,...,RIGHT_ANKLE_y,RIGHT_ANKLE_z,LEFT_ELBOW_ANGLE,RIGHT_ELBOW_ANGLE,LEFT_SHOULDER_ANGLE,RIGHT_SHOULDER_ANGLE,LEFT_HIP_ANGLE,RIGHT_HIP_ANGLE,LEFT_KNEE_ANGLE,RIGHT_KNEE_ANGLE
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,...,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,0.499777,0.995343,0.584022,0.767773,0.922266,0.308101,0.694079,0.845325,0.890492,0.475643,...,0.046323,0.539437,125.101010,98.929213,88.539268,124.316553,114.706533,119.399064,128.022738,133.522270
std,0.165426,0.024296,0.077777,0.152660,0.034965,0.124954,0.143856,0.031071,0.048523,0.158141,...,0.031228,0.175806,17.217712,18.708378,26.670369,15.892509,40.152889,44.363653,41.799609,44.890201
min,0.000000,0.798828,0.282152,0.223599,0.746521,0.000000,0.142732,0.700777,0.644011,0.190204,...,0.000000,0.197521,88.939432,52.252592,26.395024,72.010258,26.166136,22.041811,34.924074,37.644786
25%,0.433406,1.000000,0.535284,0.649035,0.912611,0.228826,0.612494,0.829630,0.871024,0.357354,...,0.027716,0.402229,112.823985,88.483322,69.910809,113.007705,87.406482,90.423067,89.205499,84.226406
50%,0.521724,1.000000,0.590505,0.790768,0.926410,0.313881,0.719988,0.848253,0.889655,0.416523,...,0.037616,0.511677,121.060060,98.974890,89.496615,127.163772,132.189418,132.544466,151.460480,159.043079
75%,0.608600,1.000000,0.635434,0.872765,0.936142,0.400958,0.795656,0.858607,0.918856,0.613500,...,0.053913,0.656381,135.406257,109.853340,109.332578,135.529237,143.349894,155.253169,160.839449,171.082518
max,0.807000,1.000000,0.761315,1.000000,1.000000,0.594716,1.000000,0.920222,1.000000,1.000000,...,0.146495,1.000000,179.230271,175.804737,134.922530,153.898301,178.670194,176.930240,173.510080,179.676564


In [30]:
df_estandar.apply(lambda x : round(x, 2)).drop_duplicates(subset=['NOSE_x', 'NOSE_y'])

,NOSE_x,NOSE_y,NOSE_z,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,RIGHT_SHOULDER_z,LEFT_ELBOW_x,...,RIGHT_ANKLE_y,RIGHT_ANKLE_z,LEFT_ELBOW_ANGLE,RIGHT_ELBOW_ANGLE,LEFT_SHOULDER_ANGLE,RIGHT_SHOULDER_ANGLE,LEFT_HIP_ANGLE,RIGHT_HIP_ANGLE,LEFT_KNEE_ANGLE,RIGHT_KNEE_ANGLE
0,0.55,1.00,0.73,0.91,0.95,0.55,0.78,0.86,0.93,0.69,...,0.03,0.24,132.03,80.59,27.87,121.68,167.60,171.90,169.78,174.43
1,0.38,0.99,0.68,0.85,1.00,0.47,0.68,0.90,0.89,0.66,...,0.03,0.32,137.06,65.36,44.53,125.77,146.06,170.09,164.97,174.11
2,0.36,0.99,0.69,0.84,1.00,0.49,0.65,0.90,0.88,0.67,...,0.03,0.29,137.22,60.99,39.77,127.93,149.69,169.61,164.06,174.57
3,0.35,0.99,0.67,0.82,1.00,0.45,0.66,0.90,0.89,0.66,...,0.03,0.34,141.00,59.79,49.27,124.41,140.95,167.93,162.32,172.43
4,0.34,0.99,0.69,0.82,1.00,0.49,0.65,0.90,0.88,0.66,...,0.04,0.28,138.65,62.13,41.58,130.69,146.65,169.95,165.44,173.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,0.28,1.00,0.58,0.74,0.97,0.28,0.56,0.90,0.86,0.66,...,0.00,0.53,128.62,66.81,79.72,117.63,141.33,161.59,150.85,162.96
371,0.70,1.00,0.67,0.91,0.93,0.43,0.86,0.85,0.92,0.50,...,0.04,0.42,145.35,103.64,69.32,118.07,156.34,143.41,160.84,165.30
372,0.71,1.00,0.70,0.92,0.93,0.47,0.85,0.85,0.95,0.51,...,0.04,0.41,154.21,104.81,66.37,108.34,156.71,153.54,162.45,176.62
373,0.72,1.00,0.76,0.93,0.93,0.56,0.84,0.85,0.97,0.51,...,0.04,0.27,159.72,115.92,56.66,102.72,167.80,169.75,167.15,176.97
